In [4]:
## 分析不同的CRE所调控的基因的表达水平
import pandas as pd

df_atac = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/CRE_atac.txt", sep="\t", names=['a', 'b','c','d','e','f', 'g', 'Geneid'])
df_h3k4me3 = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/CRE_h3k4me3.txt", sep="\t", names=['a', 'b','c','d','e','f', 'g', 'Geneid'])
df_h3k27ac = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/CRE_h3k27ac.txt", sep="\t", names=['a', 'b','c','d','e','f', 'g', 'Geneid'])
df_h3k27me3 = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/CRE_h3k27me3.txt", sep="\t", names=['a', 'b','c','d','e','f', 'g', 'Geneid'])
df_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/RNA_Seq/TM-1_allgene_FPKM_TPM.txt", sep='\t')

df_atac_gene = pd.merge(df_gene, df_atac, how='inner', on=['Geneid'])
df_h3k4me3_gene = pd.merge(df_gene, df_h3k4me3, how='inner', on=['Geneid'])
df_h3k27ac_gene = pd.merge(df_gene, df_h3k27ac, how='inner', on=['Geneid'])
df_h3k27me3_gene = pd.merge(df_gene, df_h3k27me3, how='inner', on=['Geneid'])

df_atac_gene = df_atac_gene[['Geneid', 'FPKM']]
df_h3k4me3_gene = df_h3k4me3_gene[['Geneid', 'FPKM']]
df_h3k27ac_gene = df_h3k27ac_gene[['Geneid', 'FPKM']]
df_h3k27me3_gene = df_h3k27me3_gene[['Geneid', 'FPKM']]

df_atac_gene['tags'] = ['ATAC']*df_atac_gene.shape[0]
df_h3k4me3_gene['tags'] = ['H3K4me3']*df_h3k4me3_gene.shape[0]
df_h3k27ac_gene['tags'] = ['H3K27ac']*df_h3k27ac_gene.shape[0]
df_h3k27me3_gene['tags'] = ['H3K27me3']*df_h3k27me3_gene.shape[0]

df_result = df_atac_gene.append([df_h3k4me3_gene, df_h3k27ac_gene, df_h3k27me3_gene], ignore_index=True)
df_result.drop_duplicates(keep='first', inplace=True)
print(df_result.groupby(by=['tags']).count())
df_result.to_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/CRE_regulated_gene_fpkm.txt", sep='\t', index=False)

          Geneid  FPKM
tags                  
ATAC        6683  6683
H3K27ac     2713  2713
H3K27me3    4193  4193
H3K4me3     2317  2317


In [21]:
## 分析CRE调控基因表达的差异
import pandas as pd
df_cre = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/CRE_regulated_gene_fpkm.txt", sep="\t")
df_hg_pairs = pd.read_csv("G:/TM-1_leaf_3D-genome/RNA_Seq/all_homo_gene_pairs_position.txt", sep="\t")
df_hg_pairs_id = df_hg_pairs[['geneid_at', 'geneid_dt']].copy()
df_gene_all_cre = pd.DataFrame(columns = ['geneid', 'cres'])
for index, row in df_cre.iterrows():
    g_name = row['Geneid']
    tags_list = "-".join(sorted(df_cre.query("Geneid==@g_name")['tags'].tolist()))
    df_gene_all_cre.loc[df_gene_all_cre.shape[0], ] = [g_name, tags_list]
df_gene_all_cre.drop_duplicates(keep='first', inplace=True)
df_at_cre = df_gene_all_cre.query("geneid.str.contains('Ghir_A')", engine='python')
df_dt_cre = df_gene_all_cre.query("geneid.str.contains('Ghir_D')", engine='python')
df_at_cre.columns = ['geneid_at', 'cres_at']
df_dt_cre.columns = ['geneid_dt', 'cres_dt']
df_one = pd.merge(df_hg_pairs_id, df_at_cre, how='left', on=['geneid_at'])
df_result = pd.merge(df_one, df_dt_cre, how='left', on=['geneid_dt'])
df_result.fillna("no_cres", inplace=True)
df_result.to_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/homo_gene_cres/all_homo_gene_cre_tags.txt", sep="\t", index=False)


In [61]:
## 分析有多少同源基因对受到不同的CREs调控差异， 将CREs 分为两类，1是有H3K27me3 修饰，另一类是没有H3K27me3修饰的，以此来分析调控差异
import pandas as pd
df = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/homo_gene_cres/all_homo_gene_cre_tags.txt", sep='\t')
#df_diff_cre_hg = df.query("((cres_at.str.contains('H3K27me3')) & (~cres_dt.str.contains('H3K27me3'))) | ((~cres_at.str.contains('H3K27me3')) & (cres_dt.str.contains('H3K27me3')))", engine='python')
df_diff_cre_hg = df.query("cres_at!=cres_dt", engine='python')
#df_diff_cre_hg = df.query("(cres_at=='H3K27me3' & cres_dt!='H3K27me3') | (cres_at!='H3K27me3' & cres_dt=='H3K27me3')", engine='python')
df_diff_cre_hg.to_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/homo_gene_cres/diff_cres/all_homo_gene_diff_cre.txt", sep='\t', index=False)
print(df_diff_cre_hg.shape[0])

5977


In [59]:
## 分析这些差异的CRE修饰影响了多少同源基因发生bias表达
df_bias_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_bias_expression.txt", sep="\t")
df_bias_gene.rename(columns = {'Geneid_at':'geneid_at', 'Geneid_dt':'geneid_dt'}, inplace=True)
df_diff_cre_bias_gene = pd.merge(df_bias_gene, df_diff_cre_hg, how='inner', on=['geneid_at', 'geneid_dt'])
print(df_diff_cre_bias_gene.shape[0])

716


In [64]:
## 分析随机选择的基因是否会与cres不同的修饰间存在差异
import pandas as pd
from numpy import *
df_homo_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/RNA_Seq/all_homo_gene_pairs_position.txt", sep="\t")
df_bias_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_bias_expression.txt", sep="\t")
df_bias_gene.rename(columns = {'Geneid_at':'geneid_at', 'Geneid_dt':'geneid_dt'}, inplace=True)
df_cres_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/homo_gene_cres/diff_cres/all_homo_gene_diff_cre.txt", sep="\t")
sums = []
for i in range(100):
    df_sample_gene = df_homo_gene.sample(n=1458, replace=False, axis=0)
    df_sample_bias_gene = pd.merge(df_sample_gene, df_bias_gene, how='inner', on=['geneid_at', 'geneid_dt'])
    df_diff_cres_gene_bias = pd.merge(df_sample_bias_gene, df_cres_gene, how='inner', on=['geneid_at', 'geneid_dt'])
    sums.append(df_diff_cres_gene_bias.shape[0])
print(sums)
print(mean(sums))

[43, 52, 43, 41, 33, 44, 46, 40, 49, 37, 49, 52, 48, 37, 35, 40, 40, 37, 47, 44, 42, 32, 36, 40, 47, 44, 43, 34, 46, 45, 42, 43, 40, 47, 42, 32, 45, 37, 33, 40, 37, 45, 41, 41, 37, 36, 33, 39, 36, 40, 50, 45, 49, 48, 36, 45, 40, 34, 54, 50, 44, 43, 45, 42, 44, 40, 48, 48, 44, 40, 38, 40, 42, 41, 39, 49, 41, 49, 38, 42, 35, 49, 53, 34, 47, 32, 58, 43, 40, 38, 54, 37, 31, 34, 34, 45, 36, 39, 41, 35]
41.75


In [55]:
# 由于bias 表达是质变，即需要达到某个人为设定的标准则视为它能够被认为发生了bias表达， 
# 而我们来分析这个不同修饰所能够引起的同源基因对上表达量的差异是否显著来分析复杂的CRE调控关系对于基因表达的影响
import pandas as pd
df_hg_cres = pd.read_csv('G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/homo_gene_cres/all_homo_gene_cre_tags.txt', sep='\t')
df_hg_at_activate = df_hg_cres.query("(cres_at!='H3K27me3' & cres_dt=='H3K27me3')", engine='python').copy()
df_hg_dt_activate = df_hg_cres.query("(cres_at=='H3K27me3' & cres_dt!='H3K27me3')", engine='python').copy()

df_hg_at_activate['at_sigs'] = ['activate_at']*df_hg_at_activate.shape[0]
df_hg_at_activate['dt_sigs'] = ['activate_at']*df_hg_at_activate.shape[0]

df_hg_dt_activate['at_sigs'] = ['activate_dt']*df_hg_dt_activate.shape[0]
df_hg_dt_activate['dt_sigs'] = ['activate_dt']*df_hg_dt_activate.shape[0]

df_hg_at_dt_activate = df_hg_at_activate.append(df_hg_dt_activate, ignore_index=True)
#print(df_hg_at_dt_activate)
df_hg_at = df_hg_at_dt_activate[['geneid_at', 'at_sigs']]
df_hg_dt = df_hg_at_dt_activate[['geneid_dt', 'dt_sigs']]

df_hg_fpkm = pd.read_csv('G:/TM-1_leaf_3D-genome/RNA_Seq/all_hg_expression.txt', sep='\t')
df_hg_fpkm = df_hg_fpkm[['Geneid_at', 'FPKM_at', 'Geneid_dt', 'FPKM_dt']].copy()
df_hg_fpkm.columns = ['geneid_at', 'fpkm_at', 'geneid_dt', 'fpkm_dt']
df_one = pd.merge(df_hg_fpkm, df_hg_at, how='left', on=['geneid_at'])
df_two = pd.merge(df_one, df_hg_dt, how='left', on=['geneid_dt'])
df_two.dropna(inplace=True)

df_at = df_two[['geneid_at', 'fpkm_at', 'at_sigs']].copy()
df_dt = df_two[['geneid_dt', 'fpkm_dt', 'dt_sigs']].copy()

df_at.columns = ['geneid', 'fpkm', 'tags']
df_dt.columns = ['geneid', 'fpkm', 'tags']
df_result = df_at.append(df_dt, ignore_index=True)

for index, row in df_result.iterrows():
    chrs = row['geneid']
    if 'Ghir_A' in chrs:
        subg = 'At'
    else:
        subg = 'Dt'
    df_result.loc[index, 'subg'] = subg
print(df_result)
df_result.to_csv('G:/TM-1_leaf_3D-genome/HiC/loop/loop_3k/CREs/homo_gene_cres/diff_cres/diff_cres_gene_fpkm.txt', sep='\t', index=False)



               geneid        fpkm         tags subg
0     Ghir_A01G000380    1.336266  activate_at   At
1     Ghir_A01G003790    0.232375  activate_at   At
2     Ghir_A01G004050    1.135771  activate_dt   At
3     Ghir_A01G004110    0.000000  activate_at   At
4     Ghir_A01G004130    0.936799  activate_dt   At
...               ...         ...          ...  ...
2911  Ghir_D13G022800    7.390208  activate_dt   Dt
2912  Ghir_D13G025580  114.581490  activate_dt   Dt
2913  Ghir_D13G025590    2.203940  activate_dt   Dt
2914  Ghir_D03G006480    0.000000  activate_at   Dt
2915  Ghir_D13G010850    3.922005  activate_at   Dt

[2916 rows x 4 columns]
